<a href="https://colab.research.google.com/github/HSS107048212/Overpass_turbo/blob/main/Overpass_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [309]:
!pip install geopy

In [310]:
!pip install requests

In [311]:
radius = 1500 # meter
latitude = 24.7951
longitude = 120.9960

In [312]:
import requests
def fetch_osm_data(query):
    overpass_url = "http://overpass-api.de/api/interpreter"
    response = requests.get(overpass_url, params={'data': query})
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching data from Overpass API")
        return None

# 查询要素：查询一个矩形区域内的所有餐厅
query = f"""
    [out:json];
    (
        node(around:{radius},{latitude},{longitude})["amenity"];
        node(around:{radius},{latitude},{longitude})["office"];
        node(around:{radius},{latitude},{longitude})["brand"];
        node(around:{radius},{latitude},{longitude})["tourism"];
        node(around:{radius},{latitude},{longitude})["highway"];
        node(around:{radius},{latitude},{longitude})["shop"];
    );
    out center;
    """

osm_data = fetch_osm_data(query)
if osm_data:
        # 处理返回的数据
        print(osm_data)
else:
        print("No data retrieved")



{'version': 0.6, 'generator': 'Overpass API 0.7.62.1 084b4234', 'osm3s': {'timestamp_osm_base': '2024-03-24T06:54:33Z', 'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'}, 'elements': [{'type': 'node', 'id': 26059491, 'lat': 24.7966802, 'lon': 120.9971662, 'tags': {'highway': 'traffic_signals'}}, {'type': 'node', 'id': 26069872, 'lat': 24.7933494, 'lon': 121.0013159, 'tags': {'highway': 'traffic_signals'}}, {'type': 'node', 'id': 26069885, 'lat': 24.7989655, 'lon': 120.9936149, 'tags': {'highway': 'traffic_signals', 'traffic_signals': 'signal'}}, {'type': 'node', 'id': 26069904, 'lat': 24.8003837, 'lon': 120.9944344, 'tags': {'highway': 'traffic_signals', 'traffic_signals': 'signal'}}, {'type': 'node', 'id': 26069924, 'lat': 24.8010757, 'lon': 120.9871083, 'tags': {'highway': 'traffic_signals'}}, {'type': 'node', 'id': 311080367, 'lat': 24.7910325, 'lon': 121.0045872, 'tags': {'highway': 'traffic_signals'}}, {'type':

In [313]:
# 將JSON數據轉換為DataFrame
df = pd.json_normalize(osm_data['elements'])

In [314]:
# Getting the column names of the DataFrame
#df.columns.tolist()

In [315]:
df=df[['lat','lon','tags.name','tags.amenity','tags.office','tags.brand','tags.tourism','tags.highway','tags.shop']]
df

,lat,lon,tags.name,tags.amenity,tags.office,tags.brand,tags.tourism,tags.highway,tags.shop
0,24.796680,120.997166,NaN,NaN,NaN,NaN,NaN,traffic_signals,NaN
1,24.793349,121.001316,NaN,NaN,NaN,NaN,NaN,traffic_signals,NaN
2,24.798966,120.993615,NaN,NaN,NaN,NaN,NaN,traffic_signals,NaN
3,24.800384,120.994434,NaN,NaN,NaN,NaN,NaN,traffic_signals,NaN
4,24.801076,120.987108,NaN,NaN,NaN,NaN,NaN,traffic_signals,NaN
...,...,...,...,...,...,...,...,...,...
1334,24.802733,120.995515,叁比捌手作健康餐盒,restaurant,NaN,NaN,NaN,NaN,NaN
1335,24.802691,120.995494,富國機車行,NaN,NaN,三陽機車,NaN,NaN,motorcycle
1336,24.802567,120.995428,揪好覓海鮮肉品,NaN,NaN,NaN,NaN,NaN,butcher
1337,24.802525,120.995408,傳綺編織拼布專賣店,NaN,NaN,NaN,NaN,NaN,fabric


In [316]:
# Calculate the frequency of each feature
shop_frequency = df['tags.shop'].value_counts()
df = df[df['tags.shop'].isna()]
tourism_frequency = df['tags.tourism'].value_counts()
df = df[df['tags.tourism'].isna()]
amenity_frequency = df['tags.amenity'].value_counts()
df = df[df['tags.amenity'].isna()]
office_frequency = df['tags.office'].value_counts()
df = df[df['tags.office'].isna()]
brand_frequency = df['tags.brand'].value_counts()
df = df[df['tags.brand'].isna()]
highway_frequency = df['tags.highway'].value_counts()

In [317]:
# 將它們合併成一個DataFrame
result = pd.concat([amenity_frequency, office_frequency, brand_frequency, tourism_frequency, highway_frequency,shop_frequency], axis=0)

# 重置索引，將索引轉換為列
result = result.reset_index()
# 重命名列
result = result.rename(columns={ "index" : 'Amenity', 0: 'Frequency'})
result = result.sort_values(by="Frequency", ascending=False)

result

,Amenity,Frequency
61,street_lamp,233
0,restaurant,174
62,bus_stop,141
63,traffic_signals,118
1,drinking_water,74
...,...,...
35,taxi,1
34,motorcycle_parking,1
33,music_school,1
32,lavoir,1


In [320]:
result = result[result["Amenity"] != "yes"] #來自shop的分類，發現之後，這邊直接刪掉
result

,Amenity,Frequency
61,street_lamp,233
0,restaurant,174
62,bus_stop,141
63,traffic_signals,118
1,drinking_water,74
...,...,...
35,taxi,1
34,motorcycle_parking,1
33,music_school,1
32,lavoir,1


In [291]:
# 將DataFrame輸出到Excel文件
result.to_excel(f"output{radius}_{longitude}_{latitude}.xlsx", index=False)  # 如果你不想要索引列，將index參數設置為False